In [1]:
# Installing required packages
%%capture output
!pip install langchainhub
!pip install langchain-openai
!pip install langchain
!pip install beautifulsoup4
!pip install langchain-community
!pip install faiss-cpu
!pip install -U langchain-community tavily-python
!pip gradio_client
!pip install gradio
!pip install langchain_community

# Necessary Imports
import kagglehub
import csv
import pandas as pd
import math
import numpy as np
import os
import getpass
from langchain_core.output_parsers import StrOutputParser
from google.colab import drive

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import create_openai_functions_agent

# Perform necessary imports for creating agents
from langchain import hub # Used to pull predefined prompts from LangChain Hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory  # Used to store chat history in memory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAI

In [2]:
# Setting up Open AI key
os.environ['OPENAI_API_KEY'] = getpass.getpass()


··········


In [3]:
os.environ['TAVILY_API_KEY'] = getpass.getpass()


··········


In [4]:


llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o-mini",  # or "gpt-4",
    max_tokens = 2000

    )


In [5]:
drive.mount("/content/drive")
path='/content/drive/MyDrive/IK-Company/ProjectUp/Gene/Project/MyData/dataset/'

Mounted at /content/drive


In [6]:
embeddings = OpenAIEmbeddings()

In [7]:
saved_vector = FAISS.load_local(
    path+"/faiss_index", embeddings, allow_dangerous_deserialization=True)

In [8]:
retriver = saved_vector.as_retriever()

In [9]:
# import the 'tool' decorator
from langchain.tools import tool
from langchain.tools.retriever import create_retriever_tool

@tool
def amazon_product_search(query):

  """Search for information about Amazon products.
    For any questions related to Amazon products, this tool must be used."""

  retriver_tool = create_retriever_tool(
        retriver,
        name='amazon_product_search',
        description='Search for information about Amazon products'
    )
  return retriver_tool.invoke(query)
  #return retriver.get_relevant_documents(query)


In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def search_tavily(query):
    """
    Executes a web search using the TavilySearchResults tool.

    Parameters:
        query (str): The search query entered by the user.

    Returns:
        list: A list of search results containing answers, raw content, and images.
    """
    search_tool = TavilySearchResults(
        max_results=3,
        include_answer = True,
        include_raw_content = True,
        include_images = True
    )

    return search_tool.invoke(query)


In [11]:
# hwchase17/react is a prompt template designed for ReAct-style
# conversational agents.
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [13]:
## Create a list of tools: retriever_tool and search_tool
tools = [amazon_product_search,search_tavily]

In [14]:
react_agent = create_react_agent(
    llm=llm,
    prompt=prompt,
    tools=tools
)

In [15]:
# executes the logical steps we created
agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
    )

In [16]:
res = agent_executor.invoke({"input":"What is best clothes on amazon?"})



> Entering new AgentExecutor chain...
To find the best clothes on Amazon, I should search for popular or highly-rated clothing items available on the platform. This will help me identify what is currently trending or well-reviewed. 

Action: amazon_product_search  
Action Input: "best clothes on Amazon"  Amazon Presents An Amazing Range Of Trendmode Footwear For Men/Boys Who Want To Look Perfect For Every Occasion.

Description
Make a style statement this season with this pair of trousers from Under Wrapz available on Amazon. Pair it with a chic satin blouse or a crisp cotton shirt and you are ready for the day

Explore the collection of beautifully designed Dresses & Western wear from AMARCOM on Amazon. Each piece is elegantly crafted and will surely want to add to your wardrobe. Pair this piece with heels or flats for a graceful look. Casual One Piece

A must have from our collection! We make sure your kids look stylish & cute while still being comfortable.Sort their Outfit with us

In [17]:
print(res['output'])

Some of the best clothes on Amazon include:
1. **Trousers from Under Wrapz** - Stylish and versatile, perfect for pairing with blouses or shirts.
2. **Dresses from AMARCOM** - Elegantly crafted dresses suitable for various occasions.
3. **Stylish Tops from Istyle Can** - Great for casual outings or family gatherings.
4. **Men's Slim Shirts from Amazon Brand - Inkast Denim Co.** - A fashionable choice for men.
5. **3D Printed Sweatshirts and Hoodies** - Popular among people for their cute patterns and styles, especially in fall and winter.

These items reflect a mix of styles and categories, making them some of the best clothing options available on Amazon.


In [18]:
res = agent_executor.invoke({"input":"What is current weather in Lucknow India?"})



> Entering new AgentExecutor chain...
To find the current weather in Lucknow, India, I will perform a web search for the latest weather information.  
Action: search_tavily  
Action Input: "current weather in Lucknow India"  [{'title': 'Lucknow Uttar Pradesh India 15 Day Weather Forecast - Weather Avenue', 'url': 'https://www.weatheravenue.com/en/asia/in/uttar-pradesh/lakhnau-weather-15-days.html', 'content': '☁ Lucknow Uttar Pradesh India 15 Day Weather Forecast. Today Lucknow Uttar Pradesh India: Thundery outbreaks in nearby with a temperature of 27°C and a wind North-East speed of 16 Km/h. The humidity will be 42% and there will be 0.0 mm of precipitation. ... Wednesday 16 April 2025 . Lucknow. Sunny : 42°C 108°F | 28°C 82°F : 12 Km/h 8 mph', 'score': 0.9294982}, {'title': 'Lucknow Uttar Pradesh India 15 Day Weather Forecast - Weather Avenue', 'url': 'https://m.weatheravenue.com/en/asia/in/uttar-pradesh/lakhnau-weather-15-days.html', 'content': '☼ Lucknow Uttar Pradesh India 15 Da

In [19]:
print(res['output'])

The current weather in Lucknow, India is 27°C with thundery outbreaks, 42% humidity, and a wind speed of 16 km/h from the northeast.


In [20]:
%%capture output
!pip install tiktoken

In [21]:
import tiktoken
# Example Code to see how tiktoken converts text to tokens and back
# This cell demonstrates the basic usage of tiktoken for encoding and decoding text.
encoding = tiktoken.encoding_for_model('text-embedding-3-small')
text = 'This is a small text.'
tokens = encoding.encode(text)
print(tokens)
print(len(tokens))

[2028, 374, 264, 2678, 1495, 13]
6


In [22]:
# Example usage with a list of documents:
# This cell demonstrates how to count tokens across multiple documents.
docs = [
  "This is the first document.",
  "This is the second document.",
  "And this is the third one."
]
total_tokens = 0
for doc in docs:
  tokens = encoding.encode(doc)
  total_tokens+=len(tokens)
  print(f"Tokens in document: {len(tokens)}")
print(f"Total tokens: {total_tokens}")


Tokens in document: 6
Tokens in document: 6
Tokens in document: 7
Total tokens: 19


In [23]:
#COMMENTING ON PURPOSE , SO THAT I ACCIDENTIALLY DO NOT RE-SAVE IT
#vector.save_local(path+"faiss_index")

In [24]:
new_vector = FAISS.load_local(path+"faiss_index",
                              embeddings,
                              allow_dangerous_deserialization=True)



In [25]:
similarity = new_vector.similarity_search_with_score("What is best clothes on amazon?",k=3)
similarity

[(Document(id='019b25dd-a17e-4b90-862a-b9d0d37f6138', metadata={}, page_content='Amazon Presents An Amazing Range Of Trendmode Footwear For Men/Boys Who Want To Look Perfect For Every Occasion.'),
  np.float32(0.34441003)),
 (Document(id='612a4057-07bb-483a-9fc3-d71609a6545f', metadata={}, page_content='Explore the collection of beautifully designed Dresses & Western wear from AMARCOM on Amazon. Each piece is elegantly crafted and will surely want to add to your wardrobe. Pair this piece with heels or flats for a graceful look. Casual One Piece'),
  np.float32(0.36148426)),
 (Document(id='ceb388d6-5649-4a5d-8fe6-2a1db88dbbd4', metadata={}, page_content='Description\nMake a style statement this season with this pair of trousers from Under Wrapz available on Amazon. Pair it with a chic satin blouse or a crisp cotton shirt and you are ready for the day'),
  np.float32(0.3707608))]

In [26]:
## Necessary imports
from langchain_community.callbacks import get_openai_callback


In [27]:
## Running an example query and calculating the tokens
with get_openai_callback() as cb:
   agent_executor.agent.stream_runnable = False
   response = agent_executor.invoke({"input": "How to install Backsplash Wallpaper? Also find some brands on Amazon"})
   print(f"Total Tokens: {cb.total_tokens}")
   print(f"Prompt Tokens: {cb.prompt_tokens}")
   print(f"Completion Tokens: {cb.completion_tokens}")
   print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...
To answer the question about installing backsplash wallpaper, I will first search for general instructions on how to install it. Then, I will look for brands available on Amazon. 

Action: search_tavily  
Action Input: "how to install backsplash wallpaper"  [{'title': 'How to Install Wallpaper Backsplash - Backsplash Kitchen', 'url': 'https://www.backsplashkitchen.com/how-to-install-wallpaper-backsplash/', 'content': 'Apply an even coat of heavy-duty wallpaper adhesive directly onto the backsplash surface using a paint roller or wide brush. Don’t get adhesive on the face of the wallpaper.\nFold book ends of the strips to keep edges clean and prevent sticking during application.\nAllow adhesive to become tacky according to manufacturer’s directions before applying wallpaper. This is known as “open time”.\n\nInstall the Wallpaper [...] Use a seam roller to set the seam firmly. Roll the entire strip to smooth and adhere, eliminating bubbles or wrink

In [28]:
# List of search queries
queries = [
    "Summarize the history of artificial intelligence",
    "What is the weather in London for next week? ",
    "What are the best shoes availabe in Amazon?",
    "What are some top clothing brands in Amazon?",
    "When are the next Manchester United matches scheduled?",
    "What kind of clothes should we use for travel in Switzerland next week? Check for some brands in amazon",
    "What are some of the best shoes in Amazon",
    "How to install Backsplash Wallpaper? Also find some brands on Amazon",
    "What is the current weather in Toronto?",
    "Give me a detailed summary of wedding dresses that I can buy in Amazon"
       ]

result = []
for query in queries:
  with get_openai_callback() as cb:
    agent_executor.agent.stream_runnable = False
    response = agent_executor.invoke({"input": query})



    result.append({
      "query":query,
      "Total Tokens":cb.total_tokens,
      "Prompt Tokens":cb.prompt_tokens,
      "Completion Tokens":cb.completion_tokens,
      "Total Cost (USD)":cb.total_cost
    })

    df_tokens = pd.DataFrame(result)






> Entering new AgentExecutor chain...
To summarize the history of artificial intelligence, I need to gather information about its development over the years, including key milestones, influential figures, and significant advancements. I will perform a web search to find a comprehensive overview of the history of AI.

Action: search_tavily
Action Input: "history of artificial intelligence summary"[{'title': 'History of artificial intelligence | Dates, Advances, Alan Turing, ELIZA ...', 'url': 'https://www.britannica.com/science/history-of-artificial-intelligence', 'content': 'Entertainment & Pop Culture\nLiterature\nSports & Recreation\n\nVisual Arts\n\n\nCompanions\n\nDemystified\nImage Galleries\nLists\nPodcasts\nSpotlight\nSummaries\nThe Forum\nTop Questions\n\n#WTFact\n\n\nBritannica Kids\n\n\nAsk the Chatbot Games & Quizzes History & Society Science & Tech Biographies Animals & Nature Geography & Travel Arts & Culture ProCon Money Videos\nhistory of artificial intelligence (AI)\n

In [29]:
df_tokens

,query,Total Tokens,Prompt Tokens,Completion Tokens,Total Cost (USD)
0,Summarize the history of artificial intelligence,42408,41919,489,0.006581
1,What is the weather in London for next week?,6037,5853,184,0.000988
2,What are the best shoes availabe in Amazon?,2938,2670,268,0.000561
3,What are some top clothing brands in Amazon?,4137,3826,311,0.000760
4,When are the next Manchester United matches sc...,2757,2506,251,0.000526
5,What kind of clothes should we use for travel ...,3416,3110,306,0.000650
6,What are some of the best shoes in Amazon,2901,2681,220,0.000534
7,How to install Backsplash Wallpaper? Also find...,35036,34492,544,0.004233
8,What is the current weather in Toronto?,10554,10406,148,0.001650
9,Give me a detailed summary of wedding dresses ...,2848,2239,609,0.000701


In [30]:
df_tokens.describe()

,Total Tokens,Prompt Tokens,Completion Tokens,Total Cost (USD)
count,10.000000,10.000000,10.000000,10.000000
mean,11303.200000,10970.200000,333.000000,0.001719
std,14748.681167,14666.327207,158.601527,0.002048
min,2757.000000,2239.000000,148.000000,0.000526
25%,2910.250000,2672.750000,227.750000,0.000583
50%,3776.500000,3468.000000,287.000000,0.000731
75%,9424.750000,9267.750000,444.500000,0.001484
max,42408.000000,41919.000000,609.000000,0.006581


In [31]:
df_tokens["Total Cost (USD)"].sum()

np.float64(0.0171861)

In [32]:
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0, max_tokens = 1000)
react_agent = create_react_agent(
    llm=llm,  # The OpenAI model responsible for reasoning and response generation.
    tools=tools,  # A list of external tools (e.g., web search, product retrieval).
    prompt=prompt  # The ReAct-style prompt guiding the agent's thought process.
)
# executes the logical steps we created
agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations = 10 # useful when agent is stuck in a loop
)

In [33]:
## Let's run the query again and compute its cost
with get_openai_callback() as cb:
   agent_executor.agent.stream_runnable = False
   response = agent_executor.invoke({"input": "How to install Backsplash Wallpaper? Also find some brands on Amazon"})
   print(f"Total Tokens: {cb.total_tokens}")
   print(f"Prompt Tokens: {cb.prompt_tokens}")
   print(f"Completion Tokens: {cb.completion_tokens}")
   print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new AgentExecutor chain...
To answer the question about installing backsplash wallpaper, I will first search for general instructions on how to install it. Then, I will look for brands available on Amazon. 

Action: search_tavily  
Action Input: "how to install backsplash wallpaper"  [{'title': 'How to Install Wallpaper Backsplash - Backsplash Kitchen', 'url': 'https://www.backsplashkitchen.com/how-to-install-wallpaper-backsplash/', 'content': 'Apply an even coat of heavy-duty wallpaper adhesive directly onto the backsplash surface using a paint roller or wide brush. Don’t get adhesive on the face of the wallpaper.\nFold book ends of the strips to keep edges clean and prevent sticking during application.\nAllow adhesive to become tacky according to manufacturer’s directions before applying wallpaper. This is known as “open time”.\n\nInstall the Wallpaper [...] Use a seam roller to set the seam firmly. Roll the entire strip to smooth and adhere, eliminating bubbles or wrink

In [34]:
# Creating a Prompt Template

# Pull a predefined prompt from LangChain Hub
# "hwchase17/react-chat" is a prompt template designed for ReAct-style conversational agents.
prompt = hub.pull("hwchase17/react-chat")

# Initialize a chat message history object with a session ID
# This stores the conversation history for a given session, allowing stateful interactions.
memory = ChatMessageHistory(session_id="test-session")
memory.clear()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [35]:
print(prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [36]:
llm = ChatOpenAI(model_name = 'gpt-4o-mini',temperature=0,max_tokens=1000)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
# Initialize an Agent with Chat History to maintain conversation context
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,  # The main AgentExecutor responsible for executing queries and invoking tools.

    lambda session_id: memory,

    input_messages_key="input",  # Specifies the key in the input dictionary where the user query is stored.

    history_messages_key="chat_history",  # Specifies the key under which the conversation history is stored.
)

In [38]:
# Invoke the agent with chat history
# Let's try a query based on recent data and check whether 'Tavily Search' is triggered
agent_with_chat_history.invoke(
    {"input": "What is the latest fashion trends in 2025?"},
    config={"configurable": {"session_id": "test-session"}},
)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: search_tavily
Action Input: latest fashion trends 2025[{'title': '8 Spring 2025 Fashion Trends You Can Shop and Wear Now | Vogue', 'url': 'https://www.vogue.com/article/spring-2025-fashion-trends', 'content': 'A sporty sensibility carried over from fashion’s Olympics fervor pervaded the spring/summer 2025 runways. Sabato de Sarno showed various fitted zip-up jackets at Gucci, while Bottega and Miu Miu reimagined the wind breaker—Matthieu Blazy notably lined his in plaid, a carryover from fall we’ll get into next. But other kinds of lightweight outerwear were also introduced. At Prada, Miuccia and Raf Simons showed a suede style, proof that fashion’s ongoing obsession with the texture isn’t going [...] While strong notions of femininity prevailed in the spring/summer collections, the spring/summer 2025 fashion trends we’re predicting to take over the fashion landscape are less ethereal and dreamy a

{'input': 'What is the latest fashion trends in 2025?',
 'chat_history': [],
 'output': 'The latest fashion trends for Spring/Summer 2025 emphasize practicality and empowerment, with a focus on "soft power." Key trends include:\n\n1. **Sporty Outerwear**: Lightweight jackets, especially sporty styles, are prominent, with brands like Gucci and Bottega Veneta showcasing fitted zip-up jackets and windbreakers.\n\n2. **Feminine Yet Practical Styles**: The season features strong notions of femininity, with practical designs like tailored blazers and dresses that blend comfort with style.\n\n3. **Plaid Patterns**: Plaid is making a comeback, appearing in softer colors and various styles, from dresses to outerwear.\n\n4. **Utility Fashion**: Utilitarian pieces with pockets and functional designs are in vogue, reflecting a blend of style and practicality.\n\n5. **Athleisure**: The influence of sportswear continues, with tank tops and sporty dresses being key items.\n\n6. **Pleated Skirts**: Pl

In [39]:
# Invoke the agent with chat history
# Let's try a query based on amazon data & check if 'amazon_search' tool is triggered
agent_with_chat_history.invoke(
    {"input": "Are these available on Amazon?"},
    config={"configurable": {"session_id": "test-session"}},
)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: search_tavily
Action Input: "sporty outerwear, feminine practical styles, plaid patterns, utility fashion, athleisure, pleated skirts, exaggerated proportions, colorful leather fashion 2025"
```[{'title': 'Spring 2025 Trend Report - Two of Wands', 'url': 'https://www.twoofwands.com/blog/spring-2025-trend-report', 'content': 'Sporty outerwear topped sequin dresses, dresses were featured worn over pants, and utility plaids were paired with ruffles and lace. In', 'score': 0.6307082}, {'title': 'Top 2025 Fashion Trends to Start Wearing Now', 'url': 'https://doors.nyc/blogs/news/top-2025-fashion-trend?srsltid=AfmBOoolMxcGwXgzFFkBI-_bJacJY4zLrtY2SXep-zAsFdkQkT9IgKbG', 'content': "Step into 2025's top fashion trends with sporty outerwear, bold prints, tailored power dressing, and must-have sneakers from DOORS NYC.", 'score': 0.5999311}, {'title': '10 Runway-Inspired Sporty Looks to Try Now - Vogue', 'url

{'input': 'Are these available on Amazon?',
 'chat_history': [HumanMessage(content='What is the latest fashion trends in 2025?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The latest fashion trends for Spring/Summer 2025 emphasize practicality and empowerment, with a focus on "soft power." Key trends include:\n\n1. **Sporty Outerwear**: Lightweight jackets, especially sporty styles, are prominent, with brands like Gucci and Bottega Veneta showcasing fitted zip-up jackets and windbreakers.\n\n2. **Feminine Yet Practical Styles**: The season features strong notions of femininity, with practical designs like tailored blazers and dresses that blend comfort with style.\n\n3. **Plaid Patterns**: Plaid is making a comeback, appearing in softer colors and various styles, from dresses to outerwear.\n\n4. **Utility Fashion**: Utilitarian pieces with pockets and functional designs are in vogue, reflecting a blend of style and practicality.\n\n5. **Athleisure**: The influenc